In [1]:
pip install spacy

In [2]:
import spacy
import pandas as pd
import string

In [4]:
from google.colab import drive
dados = pd.read_csv('/content/train_min.csv', encoding = 'utf-8')

In [11]:
dados.drop(dados.columns[dados.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
dados

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
5,A terrible movie as everyone has said. What ma...,0
...,...,...
595,I managed to see the MTV premiere of this movi...,1
596,I love full house so much that i couldn't live...,1
597,Scott's collection of 80's icons cannot save t...,0
598,Critics have started calling it the Oscar Winn...,0


In [12]:
pontuacao = string.punctuation
pontuacao

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
from spacy.lang.en.stop_words import STOP_WORDS
sw = STOP_WORDS
sw

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

In [14]:
reviews = dados['text']
reviews

0      I grew up (b. 1965) watching and loving the Th...
2      Why do people who do not know what a particula...
3      Even though I have great interest in Biblical ...
4      Im a die hard Dads Army fan and nothing will e...
5      A terrible movie as everyone has said. What ma...
                             ...                        
595    I managed to see the MTV premiere of this movi...
596    I love full house so much that i couldn't live...
597    Scott's collection of 80's icons cannot save t...
598    Critics have started calling it the Oscar Winn...
599    If there has ever been a worse comedy than 'Gr...
Name: text, Length: 599, dtype: object

In [15]:
pln = spacy.load('en')
pln

In [16]:
def preprocessar(texto):
  textoMinusculo = texto.lower()
  documento = pln(textoMinusculo)
  
  tokens = []
  for token in documento:
    tokens.append(token.lemma_)
  lista = []
  for palavra in tokens:
      if palavra not in sw and palavra not in pontuacao:
        lista.append(palavra)
  lista2 = ''
  for palavra in lista:
    palavra = str(palavra)
    if palavra not in '0123456789':
        lista2 += palavra + ' '
  return lista2[:-1]

In [17]:
teste = preprocessar('i LOVed 123 tHis serie so much')
teste

'love serie'

In [18]:
dados['text'] = dados['text'].apply(preprocessar)
dados['text']

0      grow b. 1965 watch love thunderbird -PRON- mat...
2      people know particular time past like feel nee...
3      great interest biblical movie bore death minut...
4      -PRON- die hard dad army fan change tape dvd a...
5      terrible movie -PRON- laugh cameo appearance s...
                             ...                        
595    manage mtv premiere movie night need tell movi...
596    love house live house -PRON- end -PRON- upset ...
597    scott collection 80 icon save teen disaster su...
598    critic start -PRON- oscar winner club understa...
599    bad comedy gray matter unaware -PRON- new york...
Name: text, Length: 599, dtype: object

In [19]:
dados

,text,label
0,grow b. 1965 watch love thunderbird -PRON- mat...,0
2,people know particular time past like feel nee...,0
3,great interest biblical movie bore death minut...,0
4,-PRON- die hard dad army fan change tape dvd a...,1
5,terrible movie -PRON- laugh cameo appearance s...,0
...,...,...
595,manage mtv premiere movie night need tell movi...,1
596,love house live house -PRON- end -PRON- upset ...,1
597,scott collection 80 icon save teen disaster su...,0
598,critic start -PRON- oscar winner club understa...,0


In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neural_network import MLPClassifier

In [22]:
vectorizer = CountVectorizer(ngram_range=(1, 1))
vectorizer.fit(dados.text)
vectorizer

CountVectorizer()

In [24]:
text_vectorizer = vectorizer.transform(dados.text)
text_vectorizer

<599x11195 sparse matrix of type '<class 'numpy.int64'>'
	with 51377 stored elements in Compressed Sparse Row format>

In [25]:
variavel = preprocessar("this movie is so crazy and good")
print(variavel)
variavel = [variavel]
variavel = vectorizer.transform(variavel)

variavel2 = preprocessar(" this movie is so bad and inappropriate")
print(variavel2)
variavel2 = [variavel2]
variavel2 = vectorizer.transform(variavel2)

movie crazy good
  movie bad inappropriate


In [75]:
dados2 = pd.read_csv('/content/test_min.csv', encoding = 'utf-8')
dados2['text'] = dados2['text'].apply(preprocessar)
text_vectorizer2 = vectorizer.transform(dados2.text)

# Regressão Logística - 76.53%

In [76]:
from sklearn.linear_model import LogisticRegression
logR = LogisticRegression(random_state=0, solver='newton-cg')

In [77]:
logR = logR.fit(text_vectorizer, dados.label)
print(logR.predict(variavel))
print(logR.predict(variavel2))

[1]
[0]


In [78]:
from sklearn.metrics import f1_score


In [79]:
previsoes = logR.predict(text_vectorizer2)
f1 = f1_score(previsoes, dados2.label)
print(f1)

0.7653061224489797


# SVM - 75.12%

In [98]:
from sklearn.svm import SVC
svm = SVC(C=3.0)

In [99]:
svm = svm.fit(text_vectorizer, dados.label)

In [100]:
previsoes = svm.predict(text_vectorizer2)
f1 = f1_score(previsoes, dados2.label)
print(f1)

0.751219512195122


# Rede Neural - 78.31% ~ 79.47%

In [101]:
from sklearn.neural_network import MLPClassifier
rede_neural = MLPClassifier(max_iter=2500,tol=0.0001)

In [102]:
rede_neural = rede_neural.fit(text_vectorizer, dados.label)

In [103]:
previsoes = rede_neural.predict(text_vectorizer2)
f1 = f1_score(previsoes, dados2.label)
print(f1)

0.783068783068783


In [104]:
from sklearn.model_selection import GridSearchCV
parametros = {'activation':('identity', 'logistic', 'tanh', 'relu'),
           'solver':('lbfgs', 'sgd', 'adam'),
            'max_iter':[1500]}
grid_search = GridSearchCV(estimator=MLPClassifier(), param_grid=parametros)
grid_search.fit(text_vectorizer, dados.label)
print(grid_search.best_params_)
print(grid_search.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_m

{'activation': 'logistic', 'max_iter': 1500, 'solver': 'adam'}
0.794733893557423
